In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook("check_particle_filter.ipynb")

# Particle filter 

This Jupyter notebook file imports (and tests) all of the code needed to do the particle filter assignment. Note that the actual code is in the .py files in this directory.

You should only have to edit particle_filter.py - you did all the "simulator" code in the previous two assignments.

Note that, although we use both sensors in this assignment, we'll only use one at a time, and update the particle distribution (importance sampling) before calling a calculate_weights method again.

Slides for this assignment: https://docs.google.com/presentation/d/1GVCAWUSUhJiHP6VBi-HusDyZmunQG1mnVlopLhtRGLs/edit?usp=sharing

In [ ]:
import sys; sys.path.insert(0, '.')
import numpy as np

# These commands will force JN to actually re-load the external file when you re-execute the import command
%load_ext autoreload
%autoreload 2

In [ ]:
from particle_filter import ParticleFilter, test_particle_filter_syntax
from world_ground_truth import WorldGroundTruth
from robot_sensors import RobotSensors
from robot_ground_truth import RobotGroundTruth

In [ ]:
# Declare variables
n_doors = 2
n_bins = 10
n_samples = 100
world_ground_truth = WorldGroundTruth()
world_ground_truth.random_door_placement(n_doors, n_bins)
robot_ground_truth = RobotGroundTruth()
robot_sensor = RobotSensors()
particle_filter = ParticleFilter()

In [ ]:
# Syntax check 1, reset probabilities
particle_filter.reset_particles(n_samples)

In [ ]:
# Syntax check 2, update move
particle_filter.update_particles_move_continuous(robot_ground_truth, 0.1)

In [ ]:
# Syntax checks 3 and 4 - the two different sensor readings
particle_filter.calculate_weights_door_sensor_reading(world_ground_truth, robot_sensor, True)
if np.isclose(np.max(particle_filter.weights), np.min(particle_filter.weights)):
    print(f"Possible error: The weights should not all be the same")

particle_filter.reset_particles(n_samples)
particle_filter.calculate_weights_distance_wall(robot_sensor, 0.1)
if np.isclose(np.max(particle_filter.weights), np.min(particle_filter.weights)):
    print(f"Possible error: The weights should not all be the same")

In [ ]:
# Syntax check 5 - importance sampling
particle_filter.resample_particles()
if not np.isclose(np.max(particle_filter.weights), np.min(particle_filter.weights)):
    print(f"Possible error: The weights should be set back to all the same")
if np.unique(particle_filter.particles, return_counts=True) == n_samples:
    print(f"Possible error: There probably should be duplicate particles {np.unique(particle_filter.particles, return_counts=True)} {n_samples}")

In [ ]:
# Syntax checks 6 and 7 - the two full updates
particle_filter.one_full_update_door(world_ground_truth, robot_ground_truth, robot_sensor, u=0.1, z=True)
particle_filter.one_full_update_distance(robot_ground_truth, robot_sensor, u=0.1, z=0.6)

In [ ]:
test_particle_filter_syntax(b_print=True)

In [ ]:
grader.check("syntax_check")

## Hours and collaborators
Required for every assignment - fill out before you hand-in.

Listing names and websites helps you to document who you worked with and what internet help you received in the case of any plagiarism issues. You should list names of anyone (in class or not) who has substantially helped you with an assignment - or anyone you have *helped*. You do not need to list TAs.

Listing hours helps us track if the assignments are too long.

In [ ]:

# List of names (creates a set)
worked_with_names = {"not filled out"}
# List of URLS (creates a set)
websites = {"not filled out"}
# Approximate number of hours, including lab/in-class time
hours = -1.5

# for all row, column in all_indices_from_where
#.   if this is the column for wrist torque 
#.      print(f"Row: {r}, Time step: {c // n_time_steps} Successful y/n: {pick_data[r, -1] == 1}, value: {pick_data[r, c]}")

In [ ]:
grader.check("hours_collaborators")

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

Submit through gradescope, particle filter. Include just this .ipynb file and ALL of the .py files

In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export(run_tests=True)